In [1]:
###### import os
import datetime
import time
import cv2
import pickle
from PIL import Image
from random import shuffle
from enum import Enum
import numpy as np
import matplotlib.pyplot as plt

import keras.backend as K
import tensorflow as tf
from keras import regularizers, initializers
from keras.models import Model
from keras.optimizers import SGD, Adam, RMSprop
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, CSVLogger, LearningRateScheduler, TerminateOnNaN

from modelDesign import MNv2_segment_depth_multiloss_model
from utils_KITTI import *

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Parameters Setting

In [5]:
# 是否要載入 pretrain weights
LOAD_DEPTH_WEIGHT = False

# 欲載入 pretrain weight 的路徑
load_depthWeight_path = ''

# 儲存 training weight 的路徑
save_depthWeight_path = '/data/mjchiu/drone/depth_prediction/model/'

# Dataset 路徑
airsim_depth_folder = '/home/mjchiu/Documents/darknet-depth/dataset/AirSim/Train'

# Model Input 解析度設定
INPUT_HEIGHT = 320
INPUT_WIDTH = 384

# 訓練的 epoch 數量
EPOCH_COUNT = 3

## Build Model

In [3]:
# 建立 Model layer 架構
model = MNv2_segment_depth_multiloss_model(
    inputShape = (INPUT_HEIGHT, INPUT_WIDTH, 3), 
    alpha = 0.35,                                           
    expansion_factor = 6,
    depth_multiplier = 1,                                        
    lock_backend_weights = False,
    CLASSES = 6,
)

# 若LOAD_DEPTH_WEIGHT為True，載入Model weight
if LOAD_DEPTH_WEIGHT:
    model.load_weights(load_depthWeight_path)

model.summary()

/usr/local/lib/python3.5/dist-packages/keras_applications/mobilenet_v2.py:315: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 384, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 [(None, 10, 12, 112) 261728      input_1[0][0]                    
__________________________________________________________________________________________________
subpixel_1 (Lambda)             (None, 20, 24, 28)   0           model_1[1][0]                    
__________________________________________________________________________________________________
conv_expand_1 (Conv2D)          (None, 20, 24, 168)  4704        subpixel_1[0][0]                 
__________________________________________________________________________________________________
conv_expan

## Training

In [6]:
# 訓練的時間戳記
timestamp = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')

# 設定訓練參數的儲存路徑與方式
model_checkpoint = ModelCheckpoint(
    filepath = save_depthWeight_path + 'airsim_' + timestamp + '_epoch-{epoch:03d}_loss-{loss:.4f}.h5',
    monitor = 'loss',
    verbose = 1,
    save_best_only = False,
    save_weights_only = True,
    mode = 'auto',
    period = 1,
)

# 設定log檔的儲存方式
csv_logger = CSVLogger(
    filename = save_depthWeight_path + 'airsim_' + timestamp + '_training_log.csv',
    separator = ',',
    append = True,
)

# 設定每個epoch的 learning rate
def lr_schedule(epoch):
    min = 0.0001
    max = 0.01
    step = EPOCH_COUNT
    if epoch > step:
        return min
    else:
        return max - epoch * (max - min) / step
learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule)

# 設定當Loss為NaN時停止訓練
terminate_on_nan = TerminateOnNaN()

# 設定loss function的callback functions
callbacks = [
    model_checkpoint,
    csv_logger,
    learning_rate_scheduler,
    terminate_on_nan,
]

In [ ]:
# 訓練起始 epoch [TODO]
initial_epoch = 0

# 訓練結束 epoch [TODO]
final_epoch = EPOCH_COUNT

# 每個 epoch 的 training iteration
steps_per_epoch = 1000

# 設定每個 loss output 的權重: [depth_pred_2x, depth_pred_4x, depth_pred_8x, depth_pred_16x,seg_pred_2x,seg_pred_4x,seg_pred_8x,seg_pred_16x]
loss_weights = [0.25, 0.25, 0.25, 0.25, 0.75, 0.75, 0.75, 0.75]

# 建立 loss function
depth_loss = custom_depth_loss(depth_weight=1.0, disparity_weight=0.0)

# 載入所有 training data 的資料路徑，建立為 list
dataList = getAirsimPath(airsim_depth_folder)

# 設定 Training data
# @param: (list) dataList: training data 的資料路徑
# @param: (list) shape: 四個 output 的 resolution
# @param: (bool) include_sky: 若為 true，則在載入GT depth map過程中，會把 depth map 天空的部分的數值設為 max 
# @param: (bool) depth_crrection: 修正PSMNet depth map 的誤差
# @param: (bool) mixGroundTruth: 是否混和 PSMNet depth map 和 sparse depth map 的深度資訊做訓練
# @param: (int) batchSize: training batch size
# @param: (int) CLASSES: 語意分割的類別數量(預設為19)
# @param: (bool) log_depth: 是否將 GT depth map 做 log transform
# @param: (bool) random_crop: 是否對 depth map 進行 random crop，作為 data augmentation
# @return: (class) data_generator
data_generator = UltraHybridGenerator_multiloss(
    dataList, 
    shape = [
        (INPUT_HEIGHT//2, INPUT_WIDTH//2),
        (INPUT_HEIGHT//4, INPUT_WIDTH//4),
        (INPUT_HEIGHT//8, INPUT_WIDTH//8),
        (INPUT_HEIGHT//16, INPUT_WIDTH//16),
    ],
    include_sky = False,
    depth_crrection = True,
    mixGroundTruth = False,
    batchSize = 5,
    CLASSES = 6,
)

# 設定 loss function
model.compile(
    optimizer = Adam(), 
    loss = [depth_loss, depth_loss, depth_loss, depth_loss, categorical_crossentropy, categorical_crossentropy, categorical_crossentropy, categorical_crossentropy],
    loss_weights = loss_weights,
)

# 開始訓練!!
history = model.fit_generator(
    generator = data_generator,
    steps_per_epoch = steps_per_epoch,
    epochs = final_epoch,
    callbacks = callbacks,
    initial_epoch = initial_epoch,
)

Total images: 39110
Epoch 1/3
1000/1000 [==============================] - 888s 888ms/step - loss: 14.9167 - depth_pw_4_loss: 14.2960 - depth_pw_3_loss: 12.7203 - depth_pw_2_loss: 12.8595 - depth_pw_1_loss: 13.5095 - activation_4_loss: 0.5306 - activation_3_loss: 0.5178 - activation_2_loss: 0.4844 - activation_1_loss: 0.5610

Epoch 00001: saving model to /data/mjchiu/drone/depth_prediction/model/airsim_2019-12-19-12-23_epoch-001_loss-14.9167.h5
Epoch 2/3
 261/1000 [======>.......................] - ETA: 11:25 - loss: 11.9436 - depth_pw_4_loss: 10.1217 - depth_pw_3_loss: 10.2688 - depth_pw_2_loss: 10.5591 - depth_pw_1_loss: 11.4848 - activation_4_loss: 0.4303 - activation_3_loss: 0.4225 - activation_2_loss: 0.4247 - activation_1_loss: 0.5025